In [4763]:
# Import Modules
import math
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from warnings import catch_warnings
from warnings import simplefilter
import scipy.stats as stats
import itertools

In [4764]:
# objective function
def objective(x):
	model20 = [13.46458095,-198.47099612,75.47320401,153.0072275803083]
	model35 = [18.66491871,-320.58644811,115.4635245,227.21147521930675]
	result = [x[0]*model20[0] + x[1]*model20[1] + x[2]*model20[2] + model20[3],x[0]*model35[0] + x[1]*model35[1] + x[2]*model35[2] + model35[3]]
	return result

In [4765]:
# evaluation function
def eval(y):
	return ((y[0] - 87.5)/87.5)**2 + ((y[1] - 76)/76)**2

In [4766]:
# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter('ignore')
        return model.predict(X, return_std=True)

In [4767]:
# Expected improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrgate score found so far
    yhat, _ = surrogate(model, X)
    score1 = np.zeros(len(yhat))
    for i in range(len(yhat)):
        score1[i] = eval(yhat[i])
    best = min(score1)

    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    score2 = np.zeros(len(mu))
    for i in range(len(mu)):
        score2[i] = eval(mu[i])
    # Calculate the expected improvement (EI)
    # Clip std to avoid division by zero
    std = np.clip(std, 1e-9, None)  # Replace None with a suitable upper bound if needed
    std2 = [i[0] for i in std]
    z = (score2 - best)**2 / std2
    ei = (score2 - best)**2 * stats.norm.cdf(z) + std2 * stats.norm.pdf(z)
    return ei

In [4768]:
# optimize the acquisition function
def opt_acquisition(X, y, model):
    # random search, generate random samples
    Xs1 = 14 + 3*np.random.random(46)
    Xs2 = 1+0.2*np.random.random(46)
    Xs3 = -0.8 + 0.4*np.random.random(46)
    Xsamples = np.asarray(list(itertools.product(Xs1,Xs2,Xs3)))
    
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = np.argmin(scores)
    return Xsamples[ix]

In [4769]:
# plot real observation vs surrogate function
def plot(X, y, model):
    # Split inputs
    Xe1 = [X[i][0] for i in range(len(X))]
    Xe2 = [X[i][1] for i in range(len(X))]
    Xe3 = [X[i][2] for i in range(len(X))]
    ye1 = [y[i][0] for i in range(len(y))]
    ye2 = [y[i][1] for i in range(len(y))]

    # scatter plot of imputs
    fig, axes = plt.subplots(2,3)
    axes[0,0].scatter(Xe1, ye1)
    axes[0,1].scatter(Xe2, ye1)
    axes[0,2].scatter(Xe3, ye1)
    axes[1,0].scatter(Xe1, ye2)
    axes[1,1].scatter(Xe2, ye2)
    axes[1,2].scatter(Xe3, ye2)

    # line plot of surragte function acorss domain
    Xp1 = np.append(Xe1,10 + 10*np.random.random(20))
    Xp2 = np.append(Xe2,2*np.random.random(20))
    Xp3 = np.append(Xe3,-10 + 20*np.random.random(20))
    Xpred = np.asarray(list(itertools.product(Xp1,Xp2,Xp3)))

    ypred, _ = surrogate(model, Xpred)
    yp1 = [i[0] for i in ypred]
    yp2 = [i[1] for i in ypred]

    Xpl1 = [i[0] for i in Xpred]
    Xpl2 = [i[1] for i in Xpred]
    Xpl3 = [i[2] for i in Xpred]

    axes[0,0].scatter(Xpl1, yp1, s=1)
    axes[0,1].scatter(Xpl2, yp1, s=1)
    axes[0,2].scatter(Xpl3, yp1, s=1)
    axes[1,0].scatter(Xpl1, yp2, s=1)
    axes[1,1].scatter(Xpl2, yp2, s=1)
    axes[1,2].scatter(Xpl3, yp2, s=1)
    # show the plot
    plt.show()

In [4774]:
# Sample Data
X = np.asarray([[np.log(8864702),1.182,-0.6144],[16.0505649,1.19114408,-0.68813221],[15.78691688,1.1436961,-0.54126377],[15.73317108,1.1070288,-0.53516194],[15.78133027,1.13980587,-0.54197605]])
y = np.asarray([[74.44,59.56],[67.06,49.637],[85.95,75.97],[91.05,84.41],[86.50,76.79]])

X1 = [X[i][0] for i in range(len(X))]
X2 = [X[i][1] for i in range(len(X))]
X3 = [X[i][2] for i in range(len(X))]
y1 = [y[i][0] for i in range(len(y))]
y2 = [y[i][1] for i in range(len(y))]

# define the model
model = GaussianProcessRegressor()
model.fit(X,y)

x = opt_acquisition(X, y, model)
actual = [67.06,49.637]
est, _ = surrogate(model, [X[2]])
print(f'>x = {x}, f()={est}, actual={actual}')
print(np.exp(x[0]),np.exp(x[0])*5)


>x = [15.8338201   1.16954276 -0.53448514], f()=[[85.95001636 75.97000649]], actual=[67.06, 49.637]
7525592.445736441 37627962.228682205


In [4771]:
# best result
score3=np.zeros(len(y))
for i in range(len(y)):
        score3[i] = eval(y[i])
ix = np.argmin(score3)
print(X[ix], y[ix])

plt.scatter(score3)

,[16.0505649,1.19114408,-0.68813221]
,[67.19,49.637]

[15.78133027  1.13980587 -0.54197605] [86.51 76.78]


<>:10: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:10: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?


TypeError: scatter() missing 1 required positional argument: 'y'